![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Adverse Drug Events Detection using Named Entity Recognition, Classification and Assertion Status Models

`ADE NER`: Extracts ADE and DRUG entities from clinical texts.

`ADE Classifier`: CLassify if a sentence is ADE-related (`True`) or not (`False`)

We use several datasets to train these models:

- Twitter dataset, which is used in paper "`Deep learning for pharmacovigilance: recurrent neural network architectures for labeling adverse drug reactions in Twitter posts`" (https://pubmed.ncbi.nlm.nih.gov/28339747/)
- ADE-Corpus-V2, which is used in paper "`An Attentive Sequence Model for Adverse Drug Event Extraction from Biomedical Text`" (https://arxiv.org/abs/1801.00625) and availe online: https://sites.google.com/site/adecorpus/home/document.
- CADEC dataset, which is sued in paper `Cadec: A corpus of adverse drug event annotations` (https://pubmed.ncbi.nlm.nih.gov/25817970)

## Healthcare NLP for Data Scientists Course

If you are not familiar with the components in this notebook, you can check [Healthcare NLP for Data Scientists Udemy Course](https://www.udemy.com/course/healthcare-nlp-for-data-scientists/) and the [MOOC Notebooks](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP) for each components.

In [0]:
from johnsnowlabs import nlp, medical
nlp.settings.enforce_versions=False

#We have already initiated the session, and the nlp.start() function is being utilized to update the credentials.
nlp.start()

In [0]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL

import os
import json
import string
import numpy as np
import pandas as pd

#from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

spark

## ADE Classifier

ADE Classifier Classifies if a sentence is ADE-related (`True`) or not (`False`)

`True` : The sentence is talking about a possible ADE

`False` : The sentences doesn't have any information about an ADE.


|index |model |Predicted Entities|
|-----:|:-----|:----------------:|
| 1| [classifierdl_ade_biobert](https://nlp.johnsnowlabs.com/2021/01/21/classifierdl_ade_biobert_en.html)   |True, False|
| 2| [classifierdl_ade_clinicalbert](https://nlp.johnsnowlabs.com/2021/01/21/classifierdl_ade_clinicalbert_en.html)  |True, False|
| 3| [classifierdl_ade_conversational_biobert](https://nlp.johnsnowlabs.com/2021/01/21/classifierdl_ade_conversational_biobert_en.html)  |True, False|
| 4| [bert_sequence_classifier_ade](https://nlp.johnsnowlabs.com/2022/02/08/bert_sequence_classifier_ade_en.html)  |True, False|
| 5| [bert_sequence_classifier_ade_augmented](https://nlp.johnsnowlabs.com/2022/07/27/bert_sequence_classifier_ade_augmented_en_3_0.html)   |ADE, noADE|
| 6| [distilbert_sequence_classifier_ade](https://nlp.johnsnowlabs.com/2022/02/08/distilbert_sequence_classifier_ade_en.html)  |True, False|
| 7| [bert_sequence_classifier_ade_augmented_v2](https://nlp.johnsnowlabs.com/2024/09/05/bert_sequence_classifier_ade_augmented_v2_en.html)  |True, False|

### ADE Classifier with BioBert

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

bert_embeddings = nlp.BertEmbeddings.pretrained("biobert_pubmed_base_cased")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")\
    .setMaxSentenceLength(512)

embeddingsSentence = nlp.SentenceEmbeddings() \
    .setInputCols(["sentence", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")\
    .setStorageRef('biobert_pubmed_base_cased')

classsifierdl = nlp.ClassifierDLModel.pretrained("classifierdl_ade_biobert", "en", "clinical/models")\
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("class")

ade_clf_pipeline = nlp.Pipeline(
    stages=[documentAssembler, 
            tokenizer,
            bert_embeddings,
            embeddingsSentence,
            classsifierdl])


empty_data = spark.createDataFrame([[""]]).toDF("text")
ade_clf_model = ade_clf_pipeline.fit(empty_data)

ade_lp_pipeline = nlp.LightPipeline(ade_clf_model)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[OK!]
classifierdl_ade_biobert download started this may take some time.
Approximate size to download 21.8 MB
[OK!]


In [0]:
text = """I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.
I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication."""

print(ade_lp_pipeline.fullAnnotate(text)[0]["class"][0].result)
print(ade_lp_pipeline.fullAnnotate(text)[0]["class"])

True
[Annotation(category, 0, 247, True, {'sentence': '0', 'False': '0.012636087', 'True': '0.9873639'}, [])]


In [0]:
text="I just took an Advil and have no gastric problems so far."

print(ade_lp_pipeline.fullAnnotate(text)[0]["class"][0].result)
print(ade_lp_pipeline.fullAnnotate(text)[0]["class"])

False
[Annotation(category, 0, 56, False, {'sentence': '0', 'False': '0.9999769', 'True': '2.3126791E-5'}, [])]


As you see `gastric problems` is not detected as `ADE` as it is in a negative context. So, classifier did a good job detecting that.

In [0]:
text="""Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. 
I had every test in the book done at the hospital, and they couldn't find anything. I was completley healthy! I am thinking it was from the voltaren. 
I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. 
I can now sleep all thru the night. I wont take this again. If I have the back pain, I will pop a tylonol instead."""

print(ade_lp_pipeline.fullAnnotate(text)[0]["class"][0].result)
print(ade_lp_pipeline.fullAnnotate(text)[0]["class"])

True
[Annotation(category, 0, 567, True, {'sentence': '0', 'False': '0.014672311', 'True': '0.9853277'}, [])]


In [0]:
ade_lp_pipeline.fullAnnotate(text)[0]["class"][0].metadata["True"]

'0.9853277'

In [0]:
texts = ["The patient was prescribed 1000 mg fish oil and multivitamins. She was discharged on zopiclone and ambrisentan.",
"I feel a bit drowsy & have a little blurred vision, after taking a pill.",
"I've been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it.",
"Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it every day for the next month to see how I get on, here goes.",
"So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50."]

for text in texts:

  result = ade_lp_pipeline.annotate(text)
  cls = ade_lp_pipeline.fullAnnotate(text)[0]["class"][0].result
  confidence = ade_lp_pipeline.fullAnnotate(text)[0]["class"][0].metadata[cls]
  print (cls,"\t",confidence)


False 	 0.9999689
True 	 0.79711705
False 	 0.99999654
True 	 0.504099
False 	 0.97631454


### ADE Classifier trained with conversational (short) sentences

This model is trained on short, conversational sentences related to ADE and is supposed to do better on the text that is short and used in a daily context.

In [0]:
conv_classsifierdl = nlp.ClassifierDLModel.pretrained("classifierdl_ade_conversational_biobert", "en", "clinical/models")\
            .setInputCols(["sentence_embeddings"]) \
            .setOutputCol("class")

conv_ade_clf_pipeline = nlp.Pipeline(
    stages=[documentAssembler, 
            tokenizer,
            bert_embeddings,
            embeddingsSentence,
            conv_classsifierdl])

empty_data = spark.createDataFrame([[""]]).toDF("text")
conv_ade_clf_model = conv_ade_clf_pipeline.fit(empty_data)

conv_ade_lp_pipeline = nlp.LightPipeline(conv_ade_clf_model)

classifierdl_ade_conversational_biobert download started this may take some time.
Approximate size to download 21.8 MB
[OK!]


In [0]:
text = "after taking a pill, he denies any pain"

conv_ade_lp_pipeline.annotate(text)['class'][0]

print(conv_ade_lp_pipeline.fullAnnotate(text)[0]["class"][0].result)
print(conv_ade_lp_pipeline.fullAnnotate(text)[0]["class"])

False
[Annotation(category, 0, 38, False, {'sentence': '0', 'False': '0.9569519', 'True': '0.04304802'}, [])]


### ADE Sequence Classifier

MedicalBertForSequenceClassification

In [0]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")
    
tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
        
sequenceClassifier = medical.BertForSequenceClassification.pretrained("bert_sequence_classifier_ade_augmented", "en", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class")
    

ade_clf_pipeline = nlp.Pipeline(stages=[
                    document_assembler, 
                    tokenizer,
                    sequenceClassifier])


data = spark.createDataFrame([["So glad I am off effexor, so sad it ruined my teeth. tip Please be carefull taking antideppresiva and read about it 1st"],
                              ["Religare Capital Ranbaxy has been accepting approval for Diovan since 2012"]]).toDF("text")
              
result = ade_clf_pipeline.fit(data).transform(data)

result.select("text", "class.result").show(truncate=False)

bert_sequence_classifier_ade_augmented download started this may take some time.
[OK!]
+-----------------------------------------------------------------------------------------------------------------------+-------+
|text                                                                                                                   |result |
+-----------------------------------------------------------------------------------------------------------------------+-------+
|So glad I am off effexor, so sad it ruined my teeth. tip Please be carefull taking antideppresiva and read about it 1st|[ADE]  |
|Religare Capital Ranbaxy has been accepting approval for Diovan since 2012                                             |[noADE]|
+-----------------------------------------------------------------------------------------------------------------------+-------+



MedicalDistilBertForSequenceClassification

In [0]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")
    
tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
    
sequenceClassifier = medical.DistilBertForSequenceClassification.pretrained("distilbert_sequence_classifier_ade", "en", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class")
 
ade_clf_pipeline = nlp.Pipeline(stages=[
                    document_assembler, 
                    tokenizer,
                    sequenceClassifier])


data = spark.createDataFrame([["I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication."],
                              ["Religare Capital Ranbaxy has been accepting approval for Diovan since 2012"]]).toDF("text")
              
result = ade_clf_pipeline.fit(data).transform(data)

result.select("text", "class.result").show(truncate=100)

distilbert_sequence_classifier_ade download started this may take some time.
[OK!]
+----------------------------------------------------------------------------------------------------+-------+
|                                                                                                text| result|
+----------------------------------------------------------------------------------------------------+-------+
|I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numb...| [True]|
|                          Religare Capital Ranbaxy has been accepting approval for Diovan since 2012|[False]|
+----------------------------------------------------------------------------------------------------+-------+



## ADE NER

Extracts `ADE` and `DRUG` entities from text.

|    | model_name                 |Predicted Entities|
|---:|:---------------------------|:----------------:|
|  1 | [ner_ade_clinical](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_clinical_en.html)      |ADE, DRUG|
|  2 | [ner_ade_clinical_v2 ](https://nlp.johnsnowlabs.com/2024/09/05/ner_ade_clinical_v2_en.html)      |ADE, DRUG, PROBLEM|
|  3 | [ner_ade_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_biobert_en.html)        |ADE, DRUG|
|  4 | [ner_ade_healthcare](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_healthcare_en.html)  |ADE, DRUG|
|  5 | [ner_ade_clinicalbert](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_clinicalbert_en.html)   |ADE, DRUG|
|  6 | [bert_token_classifier_ner_ade](https://nlp.johnsnowlabs.com/2022/01/04/bert_token_classifier_ner_ade_en.html) |ADE, DRUG|
|  7 | [bert_token_classifier_ade_tweet_binary](https://nlp.johnsnowlabs.com/2022/07/29/bert_token_classifier_ade_tweet_binary_en_3_0.html) |ADE |


#### ADE NER with Word embeddings

In [0]:
documentAssembler = nlp.DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = nlp.SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

ade_ner = medical.NerModel.pretrained("ner_ade_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = nlp.NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

ner_pipeline = nlp.Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    ade_ner,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_model = ner_pipeline.fit(empty_data)

ade_ner_lp = nlp.LightPipeline(ade_ner_model)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_ade_clinical download started this may take some time.
[OK!]


In [0]:
text = """I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.
I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication."""

light_result = ade_ner_lp.fullAnnotate(text)

chunks = []
entities = []
begin =[]
end = []
confidence = []

for n in light_result[0]['ner_chunk']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    confidence.append(n.metadata['confidence'])  

import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                    'begin': begin, 'end': end, "confidence":confidence})

df

,chunks,entities,begin,end,confidence
0,allergic reaction,ADE,10,26,0.68645
1,vancomycin,DRUG,31,40,0.9988
2,itchy skin,ADE,52,61,0.71365
3,sore throat/burning/itching,ADE,64,90,0.81229997
4,numbness of tongue,ADE,93,110,0.6829667
5,gums,ADE,116,119,0.7055


In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

visualiser.display(light_result[0], label_col='ner_chunk')

#### ADE NER with Bert embeddings

In [0]:
documentAssembler = nlp.DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = nlp.SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

bert_embeddings = nlp.BertEmbeddings.pretrained("biobert_pubmed_base_cased")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")
  
ade_ner_bert = medical.NerModel.pretrained("ner_ade_biobert", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = nlp.NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

ner_pipeline = nlp.Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    bert_embeddings,
    ade_ner_bert,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_model_bert = ner_pipeline.fit(empty_data)

ade_ner_lp_bert = nlp.LightPipeline(ade_ner_model_bert)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[OK!]
ner_ade_biobert download started this may take some time.
[OK!]


In [0]:
text = """I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.
I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication."""

light_result = ade_ner_lp_bert.fullAnnotate(text)

chunks = []
entities = []
begin =[]
end = []
confidence = []

for n in light_result[0]['ner_chunk']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    confidence.append(n.metadata['confidence'])  

import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                    'begin': begin, 'end': end, "confidence":confidence})

df

,chunks,entities,begin,end,confidence
0,allergic reaction,ADE,10,26,0.8071
1,vancomycin,DRUG,31,40,0.9979
2,itchy skin,ADE,52,61,0.94315004
3,sore throat/burning/itching,ADE,64,90,0.90735
4,numbness of tongue and gums,ADE,93,119,0.90212
5,any other medication,DRUG,227,246,0.75869995


In [0]:
visualiser.display(light_result[0], label_col='ner_chunk')

#### ADE NER for Tweets

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols("sentence")\
    .setOutputCol("token")

tokenClassifier = medical.BertForTokenClassification.pretrained("bert_token_classifier_ade_tweet_binary", "en", "clinical/models")\
    .setInputCols("token", "sentence")\
    .setOutputCol("ner")\
    .setCaseSensitive(True)

ner_converter = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")


nlpPipeline =  nlp.Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    tokenClassifier,
    ner_converter])

ade_ner_model_tweet= nlpPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

ade_ner_lp_tweet = nlp.LightPipeline(ade_ner_model_tweet)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
bert_token_classifier_ade_tweet_binary download started this may take some time.
[OK!]


In [0]:
twitter_text = """I understand you very well. :( just got 1st urgh ! humira worked for me for just 3months then got painful reactions.
This vyvanse got me sweating right now and i dont even know why!,
Wonder which drug is doing this memory lapse thing. My guess the Duloxetine.
I used to be on paxil but that made me more depressed and prozac made me angry.
Maybe it's because of the effect of seroquel, but when I eat fast carbohydrates, I feel the sugar drop."""


In [0]:
light_result = ade_ner_lp_tweet.fullAnnotate(twitter_text)

chunks = []
entities = []
begin =[]
end = []
confidence = []

for n in light_result[0]['ner_chunk']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    confidence.append(n.metadata['confidence'])  

import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                    'begin': begin, 'end': end, "confidence":confidence})

df

,chunks,entities,begin,end,confidence
0,painful reactions,ADE,98,114,0.963695
1,sweating,ADE,137,144,0.99936104
2,memory lapse,ADE,215,226,0.98965096
3,depressed,ADE,304,312,0.99975455
4,angry,ADE,333,337,0.99960786
5,sugar drop,ADE,432,441,0.9028449


In [0]:
visualiser.display(light_result[0], label_col='ner_chunk')

## ADE NER with AssertionDL Model

In [0]:
assertion_ner_converter = nlp.NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ass_ner_chunk")\
    .setWhiteList(['ADE'])

biobert_assertion = medical.AssertionDLModel.pretrained("assertion_dl_biobert", "en", "clinical/models") \
    .setInputCols(["sentence", "ass_ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

assertion_ner_pipeline = nlp.Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    bert_embeddings,
    ade_ner_bert,
    ner_converter,
    assertion_ner_converter,
    biobert_assertion])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ass_ner_model_bert = assertion_ner_pipeline.fit(empty_data)

ade_ass_ner_model_lp_bert = nlp.LightPipeline(ade_ass_ner_model_bert)

assertion_dl_biobert download started this may take some time.
[OK!]


In [0]:
import pandas as pd
text = "I feel a bit drowsy & have a little blurred vision, so far no gastric problems. I have been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it. Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it every day for the next month to see how I get on, here goes. So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50."

print (text)

light_result = ade_ass_ner_model_lp_bert.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]
confidence=[]


for n,m in zip(light_result['ass_ner_chunk'],light_result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
    confidence.append(n.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, "confidence":confidence})

df

I feel a bit drowsy & have a little blurred vision, so far no gastric problems. I have been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it. Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it every day for the next month to see how I get on, here goes. So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50.


,chunks,entities,assertion,confidence
0,drowsy,ADE,present,0.7226
1,blurred vision,ADE,present,0.9296
2,gastric problems,ADE,absent,0.8494
3,pains,ADE,absent,0.8005


In [0]:
from sparknlp_display import AssertionVisualizer

vis = AssertionVisualizer()

vis.display(light_result, 'ass_ner_chunk', 'assertion')

Looks great ! `gastric problems` is detected as `ADE` and `absent`

## ADE Relation Extraction Model

ADE Relation Extraction classifies if an adverse event is caused by a drug or not. 1: Shows the adverse event and drug entities are related, 0 : Shows the adverse event and drug entities are not related

|    | model_name                 |Predicted Entities|
|---:|:---------------------------|:----------------:|
|  1 | [re_ade_clinical](https://nlp.johnsnowlabs.com/2021/07/12/re_ade_clinical_en.html)    |0, 1|
|  2 | [re_ade_biobert](https://nlp.johnsnowlabs.com/2021/07/16/re_ade_biobert_en.html)      |0, 1|
|  3 | [redl_ade_biobert](https://nlp.johnsnowlabs.com/2021/07/12/redl_ade_biobert_en.html)  |0, 1|
|  4 | [re_ade_conversational](https://nlp.johnsnowlabs.com/2022/07/27/re_ade_cConversational_en_3_0.html)  |not_related, is_related|

In [0]:
# get relations in a pandas dataframe
import pandas as pd

def get_relations_df (results, rel_col='relations', chunk_col='ner_chunks'):
    rel_pairs=[]
    chunks = []

    for rel in results[rel_col]:
        rel_pairs.append((
            rel.metadata['entity1_begin'],
            rel.metadata['entity1_end'],
            rel.metadata['chunk1'], 
            rel.metadata['entity1'], 
            rel.metadata['entity2_begin'],
            rel.metadata['entity2_end'],
            rel.metadata['chunk2'], 
            rel.metadata['entity2'],
            rel.result, 
            rel.metadata['confidence'],
        ))

    for chunk in results[chunk_col]:
        chunks.append((
            chunk.metadata["sentence"],
            chunk.begin,
            chunk.end,
            chunk.result, 
        ))

    rel_df = pd.DataFrame(rel_pairs, columns=['entity1_begin', 'entity1_end', 'chunk1', 'entity1', 'entity2_begin', 'entity2_end', 'chunk2', 'entity2', 'relation', 'confidence'])

    chunks_df = pd.DataFrame(chunks, columns = ["sentence", "begin", "end", "chunk"])
    chunks_df.begin = chunks_df.begin.astype(str)
    chunks_df.end = chunks_df.end.astype(str)

    result_df = pd.merge(rel_df,chunks_df, left_on=["entity1_begin", "entity1_end", "chunk1"], right_on=["begin", "end", "chunk"])[["sentence"] + list(rel_df.columns)]


    return result_df

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

words_embedder = nlp.WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

pos_tagger = nlp.PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos_tags")

ner_tagger = medical.NerModel()\
    .pretrained("ner_ade_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "token", "embeddings")\
    .setOutputCol("ner_tags")    

ner_chunker = medical.NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner_tags"])\
    .setOutputCol("ner_chunks")

dependency_parser = nlp.DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos_tags", "token"])\
    .setOutputCol("dependencies")

reModel = medical.RelationExtractionModel()\
    .pretrained("re_ade_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(20)\
    .setRelationPairs(["drug-ade, ade-drug"])\
    .setRelationPairsCaseSensitive(False)\
    .setCustomLabels({"1": "is_related", "0": "not_related"})

pipeline = nlp.Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_chunker,
    dependency_parser,
    reModel
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
ade_model = pipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
ner_ade_clinical download started this may take some time.
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
re_ade_clinical download started this may take some time.
[OK!]


In [0]:
text = ["""
Hypersensitivity to aspirin can be manifested as acute asthma, urticaria and/or angioedema, or a systemic anaphylactoid reaction.
A patient had undergone a renal transplantation as a result of malignant hypertension, and immunosuppressive therapy consisting of cyclosporin and prednisone ,  developed  sweating  and  thrombosis alone 5 years following the transplantation but there were not stomach pain.  
A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for   rheumatoid arthritis  presented  with  tense bullae and cutaneous fragility on the face and the back of the hands.""",

"""We describe the side effects of 5-FU in a colon cancer patient who suffered severe mucositis,  prolonged myelosuppression, and neurologic toxicity that required admission to the intensive care unit who  has a healthy appetite.
The reported cases of in utero exposure to cyclosposphamide shared the following manifestations with our patient who suffered  growth deficiency, developmental delay, craniosynostosis, blepharophimosis, flat nasal bridge and abnormal ears.
I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.
I experienced fatigue, muscle cramps, anxiety, agression and sadness after taking Lipitor but no more adverse after passing Zocor.
A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands.""" ]

In [0]:
light_results = nlp.LightPipeline(ade_model).fullAnnotate(text)

In [0]:
get_relations_df(light_results[0], 'relations')

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,21,27,aspirin,DRUG,50,61,acute asthma,ADE,is_related,1.0
1,0,21,27,aspirin,DRUG,64,72,urticaria,ADE,is_related,1.0
2,0,21,27,aspirin,DRUG,81,90,angioedema,ADE,is_related,1.0
3,0,21,27,aspirin,DRUG,98,128,systemic anaphylactoid reaction,ADE,is_related,0.99994946
4,1,262,272,cyclosporin,DRUG,303,310,sweating,ADE,is_related,0.99999833
5,1,262,272,cyclosporin,DRUG,318,327,thrombosis,ADE,is_related,1.0
6,1,278,287,prednisone,DRUG,303,310,sweating,ADE,is_related,1.0
7,1,278,287,prednisone,DRUG,318,327,thrombosis,ADE,is_related,1.0
8,2,433,440,naproxen,DRUG,550,561,tense bullae,ADE,is_related,1.0
9,2,433,440,naproxen,DRUG,567,623,cutaneous fragility on the face and the back of the hands,ADE,is_related,0.999218


In [0]:
vis = nlp.viz.RelationExtractionVisualizer()
re_vis = vis.display(light_results[0], 'relations', show_relations=True,return_html=True) # default show_relations: True
displayHTML(re_vis)

Hypersensitivity to aspirin DRUG can be manifested as acute asthma ADE , urticaria ADE and/or angioedema ADE , or a systemic anaphylactoid reaction ADE . A patient had undergone a renal transplantation as a result of malignant hypertension, and immunosuppressive therapy consisting of cyclosporin DRUG and prednisone DRUG , developed sweating ADE and thrombosis ADE alone 5 years following the transplantation but there were not stomach pain. A 44-year-old man taking naproxen DRUG for chronic low back pain and a 20-year-old woman on oxaprozin DRUG for rheumatoid arthritis presented with tense bullae ADE and cutaneous fragility on the face and the back of the hands ADE . <polyline fill="none" points="228.0,90.0 228.2755102040816,90.66326530612244 228.55102040816323,91.32653061224487 228.82653061224494,91.98979591836735 229.10204081632654,92.6530612244898 229.37755102040816,93.31632653061226 229.65306122448976,93.97959183673468 229.9285714285715,94.64285714285717 230.2040816326531,95.30612244897961 230.4795918367347,95.96938775510206 230.7551020408163,96.63265306122447 231.03061224489792,97.29591836734693 231.30612244897955,97.95918367346938 231.5816326530612,98.62244897959184 231.85714285714286,99.28571428571429 232.13265306122452,99.94897959183676 232.40816326530617,100.6122448979592 232.6836734693878,101.27551020408167 232.95918367346937,101.93877551020408 233.23469387755102,102.60204081632654 233.5102040816326,103.26530612244898 233.78571428571425,103.92857142857142 234.0612244897959,104.59183673469389 234.33673469387753,105.25510204081633 234.6122448979592,105.9183673469388 234.8877551020408,106.58163265306125 235.16326530612244,107.24489795918367 235.4387755102041,107.90816326530613 235.71428571428572,108.57142857142858 235.98979591836735,109.23469387755104 236.26530612244898,109.89795918367348 236.54081632653063,110.56122448979593 236.81632653061223,111.22448979591837 237.09183673469386,111.88775510204083 237.36734693877548,112.55102040816327 237.64285714285714,113.21428571428572 237.9183673469388,113.87755102040816 238.19387755102042,114.54081632653063 238.46938775510205,115.20408163265307 238.74489795918367,115.86734693877551 239.0204081632653,116.53061224489795 239.29591836734693,117.19387755102042 239.57142857142858,117.85714285714286 239.84693877551018,118.5204081632653 240.12244897959184,119.18367346938776 240.39795918367346,119.84693877551021 240.67346938775512,120.51020408163265 240.94897959183675,121.1734693877551 241.22448979591837,121.83673469387756 241.5,122.5 241.5,122.5 241.7755102040816,123.16326530612244 242.05102040816325,123.82653061224487 242.3265306122449,124.48979591836735 242.60204081632654,125.1530612244898 242.87755102040813,125.81632653061224 243.1530612244898,126.47959183673467 243.42857142857147,127.14285714285717 243.7040816326531,127.8061224489796 243.97959183673473,128.46938775510205 244.2551020408163,129.13265306122446 244.5306122448979,129.7959183673469 244.80612244897955,130.45918367346937 245.08163265306118,131.1224489795918 245.35714285714286,131.78571428571428 245.63265306122452,132.44897959183675 245.90816326530614,133.1122448979592 246.18367346938777,133.77551020408166 246.4591836734694,134.43877551020407 246.73469387755102,135.10204081632654 247.0102040816326,135.76530612244895 247.28571428571428,136.42857142857142 247.5612244897959,137.09183673469386 247.83673469387756,137.7551020408163 248.1122448979592,138.41836734693877 248.3877551020408,139.0816326530612 248.6632653061224,139.74489795918365 248.9387755102041,140.40816326530611 249.2142857142857,141.07142857142856 249.48979591836735,141.73469387755102 249.76530612244898,142.39795918367346 250.0408163265306,143.06122448979593 250.31632653061223,143.72448979591834 250.5918367346939,144.38775510204079 250.86734693877548,145.05102040816325 251.14285714285717,145.71428571428572 251.41836734693877,146.37755102040816 251.69387755102042,147.0408163265306 251.96938775510205,147.70408163265304 252.24489795918367,148.36734693877548 252.52040816

## ADE models applied to Spark Dataframes

In [0]:
import pyspark.sql.functions as F

! wget -q	https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/sample_ADE_dataset.csv

In [0]:
dbutils.fs.cp("file:/databricks/driver/sample_ADE_dataset.csv", "dbfs:/")

True

In [0]:
ade_DF = spark.read\
                .option("header", "true")\
                .csv("/sample_ADE_dataset.csv")\
                .filter(F.col("label").isin(['True','False']))

ade_DF.show(truncate=50)

+--------------------------------------------------+-----+
|                                              text|label|
+--------------------------------------------------+-----+
|Do U know what Meds are R for bipolar depressio...|False|
|# hypercholesterol: Because of elevated CKs (pe...| True|
|Her weight, respirtory status and I/O should be...|False|
|* DM - Pt had several episodes of hypoglycemia ...| True|
|We report the case of a female acromegalic pati...| True|
|2 . Calcipotriene 0.005% Cream Sig: One (1) App...|False|
|Always tired, and possible blood clots. I was o...| True|
|A difference in chemical structure between thes...|False|
|10 . She was left on prednisone 20mg qd due to ...|False|
|The authors suggest that risperidone may increa...| True|
|- Per oral maxillofacial surgery there is no ev...|False|
|@marionjross Cipro is just as bad! Stay away fr...|False|
|A young woman with epilepsy had tonic-clonic se...| True|
|Intravenous methotrexate is an effective adjunc...|Fals

**With BioBert version of NER** (will be slower but more accurate)

In [0]:
import pyspark.sql.functions as F

ner_converter = nlp.NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(['ADE'])

ner_pipeline = nlp.Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    bert_embeddings,
    ade_ner_bert,
    ner_converter])


empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_model = ner_pipeline.fit(empty_data)

result = ade_ner_model.transform(ade_DF)

sample_df = result.select('text','ner_chunk.result')\
                  .toDF('text','ADE_phrases')\
                  .filter(F.size('ADE_phrases')>0).toPandas()

In [0]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)

In [0]:
sample_df.sample(20)

,text,ADE_phrases
0,# hypercholesterol: Because of elevated CKs (peaked at 819) the patient's Simvastatin was discontinued.,[elevated CKs]
18,"10mg/day for 6 years, no side effects. Recemt increase to 20mg/day caused severe pain in back, hips, legs, and feet. Unable to walk or stand for more than 10 minutes. To lower cholesterol, excercise is necessary. With Lipitor, it's impossible for me to even walk slowly, so what point is there in taking Lipitor? I stopped, MD wants me to try other statins. I have my doubts about all of them.",[severe pain in back]
28,"Lipitor is the only prescription drug I take and have taken it for four years now. I have noticed I am gaining excess weight even though I am not eating more. My stomach is bloated, with excess gas (although could be the weight gain?). I also am experiencing increased memory loss, and fatigue, but then I rarely sleep well (due to night sweats) since I went through menopause ten years ago. I think some of my symptoms are related to being post menopausal rather than from the Lipitor.","[gaining excess weight, stomach is bloated, memory loss, fatigue, night sweats]"
26,"With itraconazole, hepatotoxic reactions have only very rarely been reported, and histologic data are lacking.",[hepatotoxic reactions]
9,# thrombocytopenia: Secondary to chemotherapy and MDS/AML concerns.,[thrombocytopenia]
6,Intravenous methotrexate is an effective adjunct to steroid therapy in the treatment of steroid-resistant or life-threatening dermatomyositis-polyositis or dermatomyositis-polymyositis complicated by severe steroid-related effects.,"[life-threatening dermatomyositis-polyositis, dermatomyositis-polymyositis]"
2,We report the case of a female acromegalic patient in whom multiple hepatic adenomas appeared soon after danazol treatment for uterine fibromatosis.,[hepatic adenomas]
30,"11 . Morphine 2 mg/mL Syringe [**Month/Day (2) *]: One (1) Injection Q4H (every 4 hours) as needed for pain: hold for sedation, rr<12.","[sedation, rr<12]"
15,"As per above, ID added Atovaquone for PCP [**Name9 (PRE) *] given that Bactrim can cause BM suppression and liver damage.","[BM suppression, liver damage]"
27,Succinylcholine-induced hyperkalemia in a patient with mucositis secondary to chemotherapy.,[hyperkalemia]


**Doing the same with clinical embeddings version** (faster results)

In [0]:
import pyspark.sql.functions as F

ner_converter = nlp.NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")\
  .setWhiteList(['ADE'])

ner_pipeline = nlp.Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    ade_ner,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_model = ner_pipeline.fit(empty_data)

result = ade_ner_model.transform(ade_DF)

result.select('text','ner_chunk.result')\
.toDF('text','ADE_phrases').filter(F.size('ADE_phrases')>0)\
.show(truncate=70)


+----------------------------------------------------------------------+----------------------------------------------------------------------+
|                                                                  text|                                                           ADE_phrases|
+----------------------------------------------------------------------+----------------------------------------------------------------------+
|# hypercholesterol: Because of elevated CKs (peaked at 819) the pat...|                                                        [elevated CKs]|
|We report the case of a female acromegalic patient in whom multiple...|                                           [multiple hepatic adenomas]|
|Always tired, and possible blood clots. I was on Voltaren for about...|                      [blood clots that traveled to my eye, back pain]|
|The authors suggest that risperidone may increase affect in patient...|                                                     [increase a

### Creating sentence dataframe (one sentence per row) and getting ADE entities and categories

In [0]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = nlp.SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")\
      .setExplodeSentences(True)

tokenizer = nlp.Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

bert_embeddings = nlp.BertEmbeddings.pretrained("biobert_pubmed_base_cased")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

embeddingsSentence = nlp.SentenceEmbeddings() \
      .setInputCols(["sentence", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")\
      .setStorageRef('biobert_pubmed_base_cased')

classsifierdl = nlp.ClassifierDLModel.pretrained("classifierdl_ade_biobert", "en", "clinical/models")\
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCol("class")\
      .setStorageRef('biobert_pubmed_base_cased')

ade_ner = medical.NerModel.pretrained("ner_ade_biobert", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")
  
ner_converter = nlp.NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['ADE'])

ner_clf_pipeline = nlp.Pipeline(
    stages=[documentAssembler, 
            sentenceDetector,
            tokenizer,
            bert_embeddings,
            embeddingsSentence,
            classsifierdl,
            ade_ner,
            ner_converter])

ade_Sentences = ner_clf_pipeline.fit(ade_DF)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[OK!]
classifierdl_ade_biobert download started this may take some time.
Approximate size to download 21.8 MB
[OK!]
ner_ade_biobert download started this may take some time.
[OK!]


In [0]:
import pyspark.sql.functions as F

ade_Sentences.transform(ade_DF).select('sentence.result','ner_chunk.result','class.result')\
                               .toDF('sentence','ADE_phrases','is_ADE').show(truncate=60)

+------------------------------------------------------------+---------------------------------------------+-------+
|                                                    sentence|                                  ADE_phrases| is_ADE|
+------------------------------------------------------------+---------------------------------------------+-------+
|         [Do U know what Meds are R for bipolar depression?]|                                           []|[False]|
|         [Currently #FDA approved #quetiapine AKA #Seroquel]|                                           []|[False]|
|[# hypercholesterol: Because of elevated CKs (peaked at 8...|                               [elevated CKs]|[False]|
|[Her weight, respirtory status and I/O should be monitore...|                                           []|[False]|
|[* DM - Pt had several episodes of hypoglycemia on lantus...|                               [hypoglycemia]| [True]|
|[We report the case of a female acromegalic patient in wh...|  

## Creating a pretrained pipeline with ADE NER, Assertion and Classifer

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = nlp.Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

bert_embeddings = nlp.BertEmbeddings.pretrained("biobert_pubmed_base_cased")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

ade_ner = medical.NerModel.pretrained("ner_ade_biobert", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")\
      .setStorageRef('biobert_pubmed_base_cased')

ner_converter = nlp.NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")

assertion_ner_converter = nlp.NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ass_ner_chunk")\
      .setWhiteList(['ADE'])

biobert_assertion = medical.AssertionDLModel.pretrained("assertion_dl_biobert", "en", "clinical/models") \
      .setInputCols(["sentence", "ass_ner_chunk", "embeddings"]) \
      .setOutputCol("assertion")

embeddingsSentence = nlp.SentenceEmbeddings() \
      .setInputCols(["sentence", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")\
      .setStorageRef('biobert_pubmed_base_cased')

classsifierdl = nlp.ClassifierDLModel.pretrained("classifierdl_ade_conversational_biobert", "en", "clinical/models")\
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCol("class")

ade_clf_pipeline = nlp.Pipeline(
    stages=[documentAssembler, 
            tokenizer,
            bert_embeddings,
            ade_ner,
            ner_converter,
            assertion_ner_converter,
            biobert_assertion,
            embeddingsSentence,
            classsifierdl])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_clf_model = ade_clf_pipeline.fit(empty_data)

ade_ner_clf_pipeline = nlp.LightPipeline(ade_ner_clf_model)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[OK!]
ner_ade_biobert download started this may take some time.
[OK!]
assertion_dl_biobert download started this may take some time.
[OK!]
classifierdl_ade_conversational_biobert download started this may take some time.
Approximate size to download 21.8 MB
[OK!]


In [0]:
classsifierdl.getStorageRef()

'biobert_pubmed_base_cased'

In [0]:
text =  'I have always felt tired, but no blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldnt find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the back pain, I will pop a tylonol instead.'

light_result = ade_ner_clf_pipeline.fullAnnotate(text)

print (light_result[0]['class'][0].metadata)

chunks = []
entities = []
begin =[]
end = []
confidence = []

for n in light_result[0]['ner_chunk']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    confidence.append(n.metadata['confidence']) 

import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                    'begin': begin, 'end': end, "confidence":confidence})

df

{'sentence': '0', 'False': '0.031859856', 'True': '0.9681401'}


,chunks,entities,begin,end,confidence
0,felt tired,ADE,14,23,0.8737
1,blood clots,ADE,33,43,0.9187
2,Voltaren,DRUG,55,62,0.9984
3,stroke,ADE,116,121,0.8756
4,blood clots that traveled to my eye,ADE,131,165,0.81207144
5,voltaren,DRUG,307,314,0.8029


In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

visualiser.display(light_result[0], label_col='ner_chunk', document_col='sentence')

In [0]:
import pandas as pd

text = 'I have always felt tired, but no blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldnt find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the back pain, I will pop a tylonol instead.'

print (text)

light_result = ade_ass_ner_model_lp_bert.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]
confidence = []

for n,m in zip(light_result['ass_ner_chunk'],light_result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
    confidence.append(m.metadata['confidence']) 
        
df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, "confidence":confidence})

df

I have always felt tired, but no blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldnt find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the back pain, I will pop a tylonol instead.


,chunks,entities,assertion,confidence
0,tired,ADE,present,0.9998
1,blood clots,ADE,absent,1.0
2,stroke,ADE,present,1.0
3,blood clots that traveled to my eye,ADE,present,1.0
4,completley healthy,ADE,present,1.0


In [0]:
from sparknlp_display import AssertionVisualizer

vis = AssertionVisualizer()

vis.display(light_result, 'ass_ner_chunk', 'assertion')

In [0]:
result = ade_ner_clf_pipeline.annotate('I just took an Advil 100 mg and it made me drowsy')

print (result['class'])
print(list(zip(result['token'],result['ner'])))

['False']
[('I', 'O'), ('just', 'O'), ('took', 'O'), ('an', 'O'), ('Advil', 'B-DRUG'), ('100', 'O'), ('mg', 'O'), ('and', 'O'), ('it', 'O'), ('made', 'O'), ('me', 'O'), ('drowsy', 'B-ADE')]


In [0]:
ade_ner_clf_model.write().overwrite().save('/databricks/driver/ade_pretrained_pipeline')

In [0]:
ade_pipeline = nlp.PretrainedPipeline.from_disk('/databricks/driver/ade_pretrained_pipeline')

ade_pipeline.annotate('I just took an Advil 100 mg and it made me drowsy')

{'ner_chunk': ['Advil', 'drowsy'],
 'assertion': ['conditional'],
 'sentence_embeddings': ['I just took an Advil 100 mg and it made me drowsy'],
 'token': ['I',
  'just',
  'took',
  'an',
  'Advil',
  '100',
  'mg',
  'and',
  'it',
  'made',
  'me',
  'drowsy'],
 'ner': ['O', 'O', 'O', 'O', 'B-DRUG', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ADE'],
 'class': ['False'],
 'ass_ner_chunk': ['drowsy'],
 'embeddings': ['I',
  'just',
  'took',
  'an',
  'Advil',
  '100',
  'mg',
  'and',
  'it',
  'made',
  'me',
  'drowsy'],
 'sentence': ['I just took an Advil 100 mg and it made me drowsy']}

In [0]:
ade_pipeline.model.stages

[DocumentAssembler_6fd8903788db,
 REGEX_TOKENIZER_87bc674fc697,
 BERT_EMBEDDINGS_c6741c518b81,
 MedicalNerModel_4fc5b46ae2cf,
 NerConverter_465a6cfaf611,
 NerConverter_a25af0c332f2,
 ASSERTION_DL_2f4db8443148,
 SentenceEmbeddings_21ff28fbd40f,
 ClassifierDLModel_6edc7e323980]

## Pretrained ADE Pipeline

A pipeline for `Adverse Drug Events (ADE)` with `ner_ade_healthcare`, and `classifierdl_ade_biobert`. It will extract `ADE` and `DRUG` clinical entities, and then assign ADE status to a text(`True` means ADE, `False` means not related to ADE). Also extracts relations between `DRUG` and `ADE` entities (`1` means the adverse event and drug entities are related, `0` is not related).

In [0]:
pretrained_ade_pipeline = nlp.PretrainedPipeline('explain_clinical_doc_ade', 'en', 'clinical/models')

explain_clinical_doc_ade download started this may take some time.
Approx size to download 2 GB
[OK!]


In [0]:
pretrained_ade_pipeline.model.stages

[DocumentAssembler_bc91f7cab5f1,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_4adab43bb599,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 ENTITY_EXTRACTOR_991b113451e8,
 MedicalNerModel_4a303d875127,
 NER_CONVERTER_7ab97e03fde0,
 MedicalNerModel_12c4fe33b1c7,
 NER_CONVERTER_c507b5617de2,
 MERGE_815191b8bc5f,
 MERGE_12273ca38e67,
 CHUNKER-MAPPER_7316b33a7307,
 ASSERTION_DL_e5e007602386,
 MEDICAL_BERT_FOR_SEQUENCE_CLASSIFICATION_3c68e7367d27,
 POS_6f55785005bf,
 dependency_e7755462ba78,
 RelationExtractionModel_1fb1dfa024c7]

In [0]:
text="""The main adverse effects of Leflunomide consist of diarrhea, nausea,
liver enzyme elevation, hypertension, alopecia, and allergic skin reactions."""

result = pretrained_ade_pipeline.fullAnnotate(text)

result[0].keys()

dict_keys(['document', 'assertion', 'drug_ner_chunk', 'ner_posology_chunk', 'token', 'ner_chunks_ade', 'relations', 'ade_clinica_ner', 'ade_mappings', 'class', 'embeddings', 'pos_tags', 'dependencies', 'ner_posology', 'sentence', 'ner_ade_clinical_chunk', 'matcher_chunk'])

In [0]:
result[0]['class'][0].metadata

{'sentence': '0', 'Some(ADE)': '0.9999942', 'Some(noADE)': '5.7812E-6'}

In [0]:
text = "I experienced fatigue, muscle cramps, anxiety, agression and sadness after taking Lipitor but no more adverse after passing Zocor."

import pandas as pd

chunks = []
entities = []
begin =[]
end = []
confidence=[]

print ('sentence:', text)
print()

result = pretrained_ade_pipeline.fullAnnotate(text)

print ('ADE status:', result[0]['class'][0].result)

print ('prediction probability>> True : ', result[0]['class'][0].metadata['True'], \
        'False: ', result[0]['class'][0].metadata['False'])

for n in result[0]['ner_chunks_ade']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    confidence.append(n.metadata["confidence"])

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                'begin': begin, 'end': end,"confidence":confidence})

df


sentence: I experienced fatigue, muscle cramps, anxiety, agression and sadness after taking Lipitor but no more adverse after passing Zocor.

ADE status: ADE
prediction probability>> True :  None False:  None


,chunks,entities,begin,end,confidence
0,fatigue,ADE,14,20,0.8692
1,muscle cramps,ADE,23,35,0.9433
2,anxiety,ADE,38,44,0.9881
3,agression,ADE,47,55,0.9664
4,sadness,ADE,61,67,0.9415
5,Lipitor,DRUG,82,88,0.9988
6,Zocor,DRUG,124,128,0.9977


In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

visualiser.display(result[0], label_col='ner_chunks_ade')

#### with AssertionDL

In [0]:
import pandas as pd

text = """The side effects of 5-FU in a colon cancer patient who suffered severe mucositis, desquamating dermatitis and prolonged myelosuppression. Last week the patient experienced anterior lumbosacral radiculopathy and blurred vision after intrathecal methotrexate treatment."""

print (text)

light_result = pretrained_ade_pipeline.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]


for n,m in zip(light_result['ner_chunks_ade'],light_result['assertion']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

The side effects of 5-FU in a colon cancer patient who suffered severe mucositis, desquamating dermatitis and prolonged myelosuppression. Last week the patient experienced anterior lumbosacral radiculopathy and blurred vision after intrathecal methotrexate treatment.


,chunks,entities,assertion
0,5-FU,DRUG,Past
1,severe mucositis,ADE,Past
2,desquamating dermatitis,ADE,Past
3,myelosuppression,ADE,Past
4,anterior lumbosacral radiculopathy,ADE,Past
5,blurred vision,ADE,Past
6,methotrexate,DRUG,Past


In [0]:
from sparknlp_display import AssertionVisualizer

vis = AssertionVisualizer()

vis.display(light_result, 'ner_chunks_ade', 'assertion')

#### with Relation Extraction

In [0]:
import pandas as pd

text = """A patient had undergone a renal transplantation as a result of malignant hypertension, and immunosuppressive therapy consisting of cyclosporin and prednisone ,  developed  sweating  and  thrombosis alone 5 years following the transplantation but there were not stomach pain
"""

print (text)

results = pretrained_ade_pipeline.fullAnnotate(text)

rel_pairs=[]

for rel in results[0]["relations"]:
    rel_pairs.append((
        rel.result,
        rel.metadata['entity1'],
        rel.metadata['entity1_begin'],
        rel.metadata['entity1_end'],
        rel.metadata['chunk1'],
        rel.metadata['entity2'],
        rel.metadata['entity2_begin'],
        rel.metadata['entity2_end'],
        rel.metadata['chunk2'],
        rel.metadata['confidence']
    ))

rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])
rel_df

A patient had undergone a renal transplantation as a result of malignant hypertension, and immunosuppressive therapy consisting of cyclosporin and prednisone ,  developed  sweating  and  thrombosis alone 5 years following the transplantation but there were not stomach pain



,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,1,DRUG,131,141,cyclosporin,ADE,172,179,sweating,0.9999995
1,1,DRUG,131,141,cyclosporin,ADE,187,196,thrombosis,1.0
2,1,DRUG,147,156,prednisone,ADE,172,179,sweating,1.0
3,1,DRUG,147,156,prednisone,ADE,187,196,thrombosis,1.0


In [0]:
#from sparknlp_display import RelationExtractionVisualizer

vis = nlp.viz.RelationExtractionVisualizer()
vis.display(results[0], 'relations', show_relations=True) # default show_relations: True

## PipelineOutputParser

In [0]:
from sparknlp_jsl.pipeline_tracer import PipelineTracer

columns_directly = PipelineTracer.getParserDictDirectly("explain_clinical_doc_ade", "en", "clinical/models")
columns_directly

{'document_identifier': 'explain_clinical_doc_ade',
 'document_text': 'document',
 'entities': ['ner_chunks_ade'],
 'assertions': ['assertion'],
 'resolutions': [],
 'relations': ['relations'],
 'summaries': [],
 'deidentifications': [],
 'classifications': [{'classification_column_name': 'class',
   'sentence_column_name': 'sentence'}]}

In [0]:
from sparknlp_jsl.pipeline_tracer import PipelineTracer

pipeline_tracer = PipelineTracer(pretrained_ade_pipeline)

column_maps = pipeline_tracer.createParserDictionary()
column_maps.update({"document_identifier": "explain_clinical_doc_ade"})
column_maps

{'document_identifier': 'explain_clinical_doc_ade',
 'document_text': 'document',
 'entities': ['ner_chunks_ade'],
 'assertions': ['assertion'],
 'resolutions': [],
 'relations': ['relations'],
 'summaries': [],
 'deidentifications': [],
 'classifications': [{'classification_column_name': 'class',
   'sentence_column_name': 'sentence'}]}

In [0]:
results = pretrained_ade_pipeline.fullAnnotate(text)
results[0].keys()

dict_keys(['document', 'assertion', 'drug_ner_chunk', 'ner_posology_chunk', 'token', 'ner_chunks_ade', 'relations', 'ade_clinica_ner', 'ade_mappings', 'class', 'embeddings', 'pos_tags', 'dependencies', 'ner_posology', 'sentence', 'ner_ade_clinical_chunk', 'matcher_chunk'])

In [0]:
from sparknlp_jsl.pipeline_output_parser import PipelineOutputParser
pipeline_parser = PipelineOutputParser(column_maps)
result = pipeline_parser.run(results)

result

{'result': [{'document_identifier': 'explain_clinical_doc_ade',
   'document_id': 0,
   'document_text': ['A patient had undergone a renal transplantation as a result of malignant hypertension, and immunosuppressive therapy consisting of cyclosporin and prednisone ,  developed  sweating  and  thrombosis alone 5 years following the transplantation but there were not stomach pain\n'],
   'entities': [{'chunk_id': '372adddc',
     'chunk': 'cyclosporin',
     'begin': 131,
     'end': 141,
     'ner_label': 'DRUG',
     'ner_source': 'ner_posology_chunk',
     'ner_confidence': '0.9995'},
    {'chunk_id': '704ed794',
     'chunk': 'prednisone',
     'begin': 147,
     'end': 156,
     'ner_label': 'DRUG',
     'ner_source': 'ner_posology_chunk',
     'ner_confidence': '0.9925'},
    {'chunk_id': 'a7458c2b',
     'chunk': 'sweating',
     'begin': 172,
     'end': 179,
     'ner_label': 'ADE',
     'ner_source': 'ner_ade_clinical_chunk',
     'ner_confidence': '0.729'},
    {'chunk_id': '8

You can check the links below if you want to see more examples of; 

You can check the links below if you want to see more examples of;

- Pretrained Clinical Pipelines in Spark NLP : [07.0.Pretrained Clinical Pipelines](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/07.0.Pretrained_Clinical_Pipelines.ipynb)
- Task Based Clinical Pipelines in Spark NLP : [07.2. Task Based Clinical Pretrained Pipelines](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/07.2.Task_Based_Clinical_Pretrained_Pipelines.ipynb)
- PipelineTracer and PipelineOutputParser in Spark NLP : [07.4.PipelineTracer and PipelineOutputParser](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/07.4.PipelineTracer_and_PipelineOutputParser.ipynb)
- Relation Extraction Model of DRUG and ADE entities `re_ade_biobert`: [03.0 Clinical Relation Extraction Model](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/03.0.Clinical_Relation_Extraction.ipynb)